# Leitura de nota de corretagem

É necessário que o arquivo seja convertido em xml

In [4]:
pip install correpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import io

from correpy.parsers.brokerage_notes.b3_parser.b3_parser import B3Parser

import numpy as np
import pandas as pd


In [6]:
with open('/kaggle/input/btg-notas-de-corretagem-2022/BTG notas de corrertagem 2022.pdf', 'rb') as f:
    content = io.BytesIO(f.read())
    content.seek(0)
    
    brokerage_notes = B3Parser(brokerage_note=content).parse_brokerage_note()
    

In [7]:
notas = pd.DataFrame(brokerage_notes)
notas.head()

,reference_date,settlement_fee,registration_fee,term_fee,ana_fee,emoluments,operational_fee,execution,custody_fee,taxes,others,transactions
0,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,"[{'transaction_type': TransactionType.BUY, 'am..."
1,2022-05-24,0.01,0.04,0.00,0.00,0.02,0,0.00,0,0,0,"[{'transaction_type': TransactionType.BUY, 'am..."
2,2022-05-25,0.00,0.02,0.00,0.00,0.01,0,0.00,0,0,0,"[{'transaction_type': TransactionType.BUY, 'am..."
3,2022-05-27,0.13,0.33,0.00,0.00,0.17,0,0.00,0,0,0,"[{'transaction_type': TransactionType.SELL, 'a..."
4,2022-05-30,0.04,0.11,0.00,0.00,0.06,0,0.00,0,0,0,"[{'transaction_type': TransactionType.SELL, 'a..."


## Normalização da coluna transactions

Note-se que a coluna `transactions` é composta por um dicionário.

Nos próximos passos, iremos abrir e incorporar as colunas derivadas ao DataFrame `notas`.

In [8]:
len(notas) #mesma quantidade de notas

77

In [9]:
transactions_exploded_df = notas[['transactions']].explode('transactions')
len(transactions_exploded_df)

164

In [10]:
print(len(transactions_exploded_df))
transactions_exploded_df.head()

164


,transactions
0,"{'transaction_type': TransactionType.BUY, 'amo..."
0,"{'transaction_type': TransactionType.SELL, 'am..."
0,"{'transaction_type': TransactionType.BUY, 'amo..."
0,"{'transaction_type': TransactionType.BUY, 'amo..."
1,"{'transaction_type': TransactionType.BUY, 'amo..."


In [11]:
print(type(transactions_exploded_df.iloc[0,0]))
transactions_exploded_df.iloc[0,0]

<class 'dict'>


{'transaction_type': <TransactionType.BUY: 'buy'>,
 'amount': 200,
 'unit_price': Decimal('0.78'),
 'security': {'name': 'DE VENDA 07/22 BBASS363 ON', 'ticker': 'BASS36'},
 'source_withheld_taxes': Decimal('0')}

In [12]:
transactions_normalized_df = pd.json_normalize(transactions_exploded_df['transactions'])
transactions_normalized_df.index = transactions_exploded_df.index
transactions_normalized_df = transactions_normalized_df.rename(columns={'security.name': 'security_name', 'security.ticker': 'security_ticker'})
transactions_normalized_df.head(15)

#tem q separar por índice

,transaction_type,amount,unit_price,source_withheld_taxes,security_name,security_ticker
0,TransactionType.BUY,200,0.78,0,DE VENDA 07/22 BBASS363 ON,BASS36
0,TransactionType.SELL,200,1.18,0.01,DE COMPRA 06/22 BBDCF209 PN,None
0,TransactionType.BUY,110,1.39,0,DE VENDA 05/22 BBDCQ221 PN,None
0,TransactionType.BUY,100,0.10,0,DE COMPRA 06/22 EMBRF160 ON,None
1,TransactionType.BUY,200,0.31,0,DE VENDA 10/22 PETRV30 PN,ETRV3
2,TransactionType.BUY,100,0.31,0,DE VENDA 06/22 ITSAR987 PN,None
3,TransactionType.SELL,100,4.81,0.02,DE VENDA 10/22 VALEV867 ON,None
4,TransactionType.SELL,100,1.72,0.01,DE VENDA 06/22 SULAR285 UNT,None
5,TransactionType.SELL,100,0.72,0.00,DE COMPRA 07/22 EMBRG140 ON,None
5,TransactionType.SELL,100,0.64,0.00,DE COMPRA 06/22 SAPRF200 UNT,None


In [13]:
notas = pd.merge(left = notas, right = transactions_normalized_df, how='right', left_index=True, right_index=True)
notas.reset_index(inplace=True, drop=True)
notas.head()

,reference_date,settlement_fee,registration_fee,term_fee,ana_fee,emoluments,operational_fee,execution,custody_fee,taxes,others,transactions,transaction_type,amount,unit_price,source_withheld_taxes,security_name,security_ticker
0,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,"[{'transaction_type': TransactionType.BUY, 'am...",TransactionType.BUY,200,0.78,0,DE VENDA 07/22 BBASS363 ON,BASS36
1,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,"[{'transaction_type': TransactionType.BUY, 'am...",TransactionType.SELL,200,1.18,0.01,DE COMPRA 06/22 BBDCF209 PN,None
2,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,"[{'transaction_type': TransactionType.BUY, 'am...",TransactionType.BUY,110,1.39,0,DE VENDA 05/22 BBDCQ221 PN,None
3,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,"[{'transaction_type': TransactionType.BUY, 'am...",TransactionType.BUY,100,0.10,0,DE COMPRA 06/22 EMBRF160 ON,None
4,2022-05-24,0.01,0.04,0.00,0.00,0.02,0,0.00,0,0,0,"[{'transaction_type': TransactionType.BUY, 'am...",TransactionType.BUY,200,0.31,0,DE VENDA 10/22 PETRV30 PN,ETRV3


In [14]:
notas.security_ticker

0      BASS36
1        None
2        None
3        None
4       ETRV3
        ...  
159    ETRB32
160      None
161    IIAM35
162    IIAX35
163      None
Name: security_ticker, Length: 164, dtype: object

In [15]:
notas.pop('transactions')

0      [{'transaction_type': TransactionType.BUY, 'am...
1      [{'transaction_type': TransactionType.BUY, 'am...
2      [{'transaction_type': TransactionType.BUY, 'am...
3      [{'transaction_type': TransactionType.BUY, 'am...
4      [{'transaction_type': TransactionType.BUY, 'am...
                             ...                        
159    [{'transaction_type': TransactionType.SELL, 'a...
160    [{'transaction_type': TransactionType.SELL, 'a...
161    [{'transaction_type': TransactionType.SELL, 'a...
162    [{'transaction_type': TransactionType.SELL, 'a...
163    [{'transaction_type': TransactionType.SELL, 'a...
Name: transactions, Length: 164, dtype: object

In [16]:
notas = notas.rename(columns={'reference_date': 'data_op', 
                      'settlement_fee': 'tx_liq', 
                      'registration_fee': 'tx_regist', 
                      'term_fee': 'tx_termo', 
                      'ana_fee': 'tx_ana', 
                      'emoluments': 'emolumentos',
                     'operational_fee': 'tx_oper',
                     'execution':'execucao',
                     'custody_fee': 'tx_custodia',
                     'taxes': 'impostos',
                     'others': 'outros',
                     'transaction_type': 'tipo_transac',
                     'amount': 'quant',
                     'unit_price':'preco_unitario',
                     'source_withheld_taxes': 'irrf_fonte',
                     'security_name':'acao'})
notas.head()

,data_op,tx_liq,tx_regist,tx_termo,tx_ana,emolumentos,tx_oper,execucao,tx_custodia,impostos,outros,tipo_transac,quant,preco_unitario,irrf_fonte,acao,security_ticker
0,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,TransactionType.BUY,200,0.78,0,DE VENDA 07/22 BBASS363 ON,BASS36
1,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,TransactionType.SELL,200,1.18,0.01,DE COMPRA 06/22 BBDCF209 PN,None
2,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,TransactionType.BUY,110,1.39,0,DE VENDA 05/22 BBDCQ221 PN,None
3,2022-05-20,0.15,0.38,0.00,0.00,0.20,0,0.00,0,0,0,TransactionType.BUY,100,0.10,0,DE COMPRA 06/22 EMBRF160 ON,None
4,2022-05-24,0.01,0.04,0.00,0.00,0.02,0,0.00,0,0,0,TransactionType.BUY,200,0.31,0,DE VENDA 10/22 PETRV30 PN,ETRV3


In [27]:
notas['tipo_transac2'] = notas['tipo_transac'].apply(lambda x: 'compra' if ('TransactionType.BUY' == str(x)) else 'venda')
notas[['tipo_transac','tipo_transac2']].sample(n=10)

,tipo_transac,tipo_transac2
71,TransactionType.BUY,compra
33,TransactionType.SELL,venda
3,TransactionType.BUY,compra
117,TransactionType.SELL,venda
37,TransactionType.BUY,compra
132,TransactionType.BUY,compra
114,TransactionType.SELL,venda
113,TransactionType.BUY,compra
10,TransactionType.SELL,venda
30,TransactionType.SELL,venda


In [63]:
#notas.pop('tipo_transac')
notas['ticker'] = notas['acao'].apply(lambda x: str(x.split()[3]))
notas['tipo_opcao'] = notas['acao'].apply(lambda x: 'put' if str(x.split()[1]) == 'VENDA' else ('call' if str(x.split()[1]) == 'COMPRA' else 'None'))
notas['vencimento_opcao'] = notas['acao'].apply(lambda x: str(x.split()[2]))
notas['total_operacao'] = notas['quant'] * notas['preco_unitario']
#não funcionou
notas['total_operacao'] = notas['total_operacao'].apply(lambda y: y*-1 if str(notas['tipo_transac2']) == 'venda' else y)
notas[['acao','ticker','vencimento_opcao','tipo_opcao','tipo_transac2','quant','preco_unitario','total_operacao']]


,acao,ticker,vencimento_opcao,tipo_opcao,tipo_transac2,quant,preco_unitario,total_operacao
0,DE VENDA 07/22 BBASS363 ON,BBASS363,07/22,put,compra,200,0.78,156.00
1,DE COMPRA 06/22 BBDCF209 PN,BBDCF209,06/22,call,venda,200,1.18,236.00
2,DE VENDA 05/22 BBDCQ221 PN,BBDCQ221,05/22,put,compra,110,1.39,152.90
3,DE COMPRA 06/22 EMBRF160 ON,EMBRF160,06/22,call,compra,100,0.10,10.00
4,DE VENDA 10/22 PETRV30 PN,PETRV30,10/22,put,compra,200,0.31,62.00
...,...,...,...,...,...,...,...,...
159,DE COMPRA 02/23 PETRB32 PN,PETRB32,02/23,call,compra,100,1.32,132.00
160,DE VENDA 02/23 PETRN270 PN,PETRN270,02/23,put,venda,200,2.59,518.00
161,DE VENDA 01/23 VIIAM350 ON,VIIAM350,01/23,put,venda,400,1.50,600.00
162,DE VENDA 12/22 VIIAX350 ON,VIIAX350,12/22,put,compra,400,1.55,620.00
